In [2]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from PytorchNN import DEVICE, Network

import PytorchNN as net

Using device mps


In [4]:
# Dataset Loading
df = pd.read_csv("data/chess_data_output-110524.csv")
input_features = df.iloc[:, :10].values
output_labels = df.iloc[:, 10:].values
output_labels

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]])

In [5]:
# Tensor Conversion
inputs_tensor = torch.tensor(input_features, dtype=torch.float32)
outputs_tensor = torch.tensor(output_labels, dtype=torch.float32)

# Outputs -> Class Indices
outputs_tensor = torch.argmax(outputs_tensor, dim=1)

dataset = TensorDataset(inputs_tensor, outputs_tensor)

batch_size = 20
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [63]:
# Model creation
model = net.Network().to(DEVICE)

In [64]:
%%time
model.train_model(train_loader, epochs=100)

Epoch 1 / 100 - Loss: 1.0642
Epoch 6 / 100 - Loss: 1.0235
Epoch 11 / 100 - Loss: 1.0116
Epoch 16 / 100 - Loss: 1.0082
Epoch 21 / 100 - Loss: 1.0065
Epoch 26 / 100 - Loss: 1.0049
Epoch 31 / 100 - Loss: 1.0037
Epoch 36 / 100 - Loss: 1.0023
Epoch 41 / 100 - Loss: 1.0007
Epoch 46 / 100 - Loss: 0.9995
Epoch 51 / 100 - Loss: 0.9979
Epoch 56 / 100 - Loss: 0.9968
Epoch 61 / 100 - Loss: 0.9964
Epoch 66 / 100 - Loss: 0.9950
Epoch 71 / 100 - Loss: 0.9937
Epoch 76 / 100 - Loss: 0.9931
Epoch 81 / 100 - Loss: 0.9929
Epoch 86 / 100 - Loss: 0.9922
Epoch 91 / 100 - Loss: 0.9917
Epoch 96 / 100 - Loss: 0.9911


In [87]:
model.test_model(train_loader)

AttributeError: 'Network' object has no attribute 'test_model'

In [6]:
with torch.no_grad():
    r = model.forward(torch.tensor([
        [8, 2, 2, 2, 1, 
         8, 2, 2, 2, 1]
    ]).type(torch.float32).to(DEVICE))
r

tensor([[0.5720, 0.3540, 0.0740]], device='mps:0')

In [83]:
import json
# Save the trained weights to JSON
model_weights = {key: value.detach().cpu().numpy().tolist() for key, value in model.state_dict().items()}

with open('exports/model_weights-100epoch-Adam-94k-102424.json', 'w') as f:
    json.dump(model_weights, f)

In [84]:
# Save model with serialization
torch.save(model.state_dict(), "exports/model_state_dict-100epoch-Adam-94k-102424.pt")

In [4]:
# Load model with serialization
model = Network().to(DEVICE)
model.load_state_dict(torch.load("exports/model_state_dict-100epoch-Adam-94k-102424.pt", weights_only=True))
model.eval()

Network(
  (network_layers): Sequential(
    (0): Linear(in_features=10, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=3, bias=True)
    (5): Softmax(dim=1)
  )
  (loss_func): CrossEntropyLoss()
)